In [0]:
#!/usr/bin/env python
import pandas as pd
from sodapy import Socrata

# client = Socrata("www.datos.gov.co", None)

token = dbutils.secrets.get("claves","token_app")
codigo_dataset = dbutils.widgets.get("codigo_dataset")
# Example authenticated client (needed for non-public datasets):
client = Socrata('www.datos.gov.co',str(token),timeout=2000)

In [0]:
import time
# 3. Parámetros de paginación
limit = 500000
offset = 0
write_mode = "overwrite"
reintentos=5

print(f"Iniciando carga por lotes para el dataset: {codigo_dataset}")

# 4. Bucle para obtener y cargar los datos por lotes
while True:
    intentos = 0

    while intentos < reintentos:
        try:
            print(f"Obteniendo y cargando lote {offset}...")
            # Construye y ejecuta la consulta para el lote actual
            query = f"SELECT numero_del_contrato, numero_de_proceso, nit_de_la_entidad, documento_proveedor, estado_del_proceso LIMIT {limit} OFFSET {offset}"
            results = client.get(codigo_dataset, query=query)

            # Si la API no devuelve más registros, se termina el bucle
            if not results:
                print("Carga de datos finalizada.")
                break

            # Convierte el lote a un DataFrame de Spark y lo escribe en la tabla Delta
            spark.createDataFrame(results).write \
                .format("delta") \
                .mode(write_mode) \
                .option("overwriteSchema", "true") \
                .saveAsTable("diplomado_datos.ids_contratos_procesos")

            print(f"Lote de {len(results)} registros desde offset {offset} cargado.")

            # Se cambia a modo 'append' para las siguientes iteraciones y se incrementa el offset
            write_mode = "append"
            offset += limit
            break

        except Exception as e:
            intentos +=1
            print("Error al obtener o cargar el lote:", e)
            print(f"Intento {intentos} de {reintentos}...")
            time.sleep(60)
    else:
        print("Se alcanzó el número máximo de intentos. Terminando la carga.")
        break

In [0]:
df_secop_id=spark.table("diplomado_datos.ids_contratos_procesos")
df_secop_id.count()